## Overview

To use this utility:

* Add an IBM Cloud API key or your IBM Cloud credentials in the next cell. No further changes are required.
* Run all cells (**Cell** > **Run all**) (This will take about 45 seconds.)
* Scroll to the bottom of the notebook and try the app

> **Note: IAM managed services (e.g. Cloud Object Storage and Streaming Analytics) currently cannot be discovered**

In [ ]:
#@ hidden_cell
# Specify IBM Cloud API key (recommended)
api_key = None

# ... or a user id/password
id = None
pw = None

This notebook requires pixiedust version 1.1.8 or above. Upgrade if necessary:

In [ ]:
!pip install -U pixiedust

In [ ]:
import json
import pandas as pd
import requests
import urllib

In [ ]:
API_base_URL = 'https://api.ng.bluemix.net{}'
debug = True

#### Obtain access token

In [ ]:
if api_key is not None:
    id = 'apikey'
    pw = api_key

response = requests.get(API_base_URL.format('/info'))
results = response.json()
auth_endpoint = results['authorization_endpoint'] + '/oauth/token'

data = 'grant_type=password&username={0}&password={1}'.format(id, pw)
auth = ('cf', '')
headers = {
    'accept': 'application/json',
    'content-type': 'application/x-www-form-urlencoded;charset=utf-8'
    }
response = requests.post(auth_endpoint, data=data, headers=headers, auth=auth)
if response.status_code == 200:
    results = response.json()
    authorization = results['token_type'] + ' ' + results['access_token']
    print('Got OAuth token')
else:
    print('Fatal error obtaining authentication token: {}'.format(response))


In [ ]:
url = API_base_URL.format('/v2/organizations')

http_headers = {
    'accept': 'application/json',
    'content-type': 'application/json',
    'authorization': authorization
    }
response = requests.get(url, headers=http_headers)
if response.status_code == 200:
    orgs = []
    for org in json.loads(response.text)['resources']:
        orgs.append(org['entity']['name'])   
    orgs = sorted(orgs, key=lambda s: s.lower())
    print('You have access to the following organizations: {}'.format(orgs))
    service_plans = None
else:
    print('Fatal error retrieving organizations: {}'.format(response))

In [ ]:
org_lookup = {}

def fetch_organization_list(url):
    http_headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'authorization': authorization
        }
    if debug:
        print('GET {}'.format(url))
    response = requests.get(API_base_URL.format(url), headers=http_headers)
    if response.status_code == 200:
        for resource in response.json().get('resources', []):
            org_lookup[resource['metadata']['guid']] = resource['entity']['name']          
        return response.json()['next_url']
    else:
        print('Fatal error retrieving organization list: {}'.format(response))
        return None

# https://apidocs.cloudfoundry.org/280/services/list_all_services.html
url = '/v2/organizations?results-per-page=100'

while url is not None:
    print('Populating organization lookup...')
    url = fetch_organization_list(url)
    
print('Found {} organizations.'.format(len(org_lookup)))   

In [ ]:
space_lookup = {}

def fetch_space_list(url):
    http_headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'authorization': authorization
        }
    if debug:
        print('GET {}'.format(url))
    response = requests.get(API_base_URL.format(url), headers=http_headers)
    if response.status_code == 200:
        for resource in response.json().get('resources', []):
                space_lookup[resource['metadata']['guid']] = {
                    'space_name': resource['entity']['name'],
                    'org_guid': resource['entity']['organization_guid'],
                    'org_name': org_lookup[resource['entity']['organization_guid']]
                }
        return response.json()['next_url']
    else:
        print('Fatal error retrieving organization list: {}'.format(response))
        return None

# https://apidocs.cloudfoundry.org/280/organizations/list_all_spaces_for_the_organization.html


for org_guid in org_lookup.keys(): 
    url = '/v2/organizations/{}/spaces?results-per-page=100'.format(org_guid)
    while url is not None:
        print('Searching for spaces in org {}...'.format(org_lookup[org_guid]))
        url = fetch_space_list(url)
        
del org_guid        
    
print('Found {} spaces.'.format(len(space_lookup)))   

In [ ]:
service_lookup = {}

def fetch_complete_service_list(url):
    http_headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'authorization': authorization
        }
    if debug:
        print('GET {}'.format(url))
    response = requests.get(API_base_URL.format(url), headers=http_headers)
    if response.status_code == 200:
        for resource in response.json().get('resources', []):
            service_lookup[resource['metadata']['guid']] = resource['entity']['label']          
        return response.json()['next_url']
    else:
        print('Fatal error retrieving service list: {}'.format(response))
        return None

# https://apidocs.cloudfoundry.org/280/services/list_all_services.html
url = '/v2/services?results-per-page=100'

while url is not None:
    print('Populating service lookup...')
    url = fetch_complete_service_list(url)
    
print('Found {} services.'.format(len(service_lookup)))    

In [ ]:
service_plan_lookup = {}

def fetch_service_plan_list(url):
    http_headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'authorization': authorization
        }
    if debug:
        print('GET {}'.format(url))
    response = requests.get(API_base_URL.format(url), headers=http_headers)
    if response.status_code == 200:
        for resource in response.json().get('resources', []):
            service_plan_lookup[resource['metadata']['guid']] = resource['entity']['name']          
        return response.json()['next_url']                
    else:
        print('Fatal error retrieving service plan information: {}'.format(response))
        return None

# https://apidocs.cloudfoundry.org/280/service_plans/list_all_service_plans.html
url = '/v2/service_plans?results-per-page=100'

while url is not None:
    print('Loading service plan information...')
    url = fetch_service_plan_list(url)
    
print('Found {} service plans.'.format(len(service_plan_lookup)))    


In [ ]:
service_instances = []

def fetch_service_instance_list(url):
    http_headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'authorization': authorization
        }
    
    if debug:
        print('GET {}'.format(url))
    response = requests.get(API_base_URL.format(url), headers=http_headers)
    if response.status_code == 200:     
        for resource in response.json().get('resources', []):
                service = {'service_instance_name':resource['entity']['name'],
                           'service_instance_guid':resource['metadata']['guid'],
                           'service_guid':resource['entity']['service_guid'],
                           'created_at':resource['metadata']['created_at'],
                           'service_plan_guid': resource['entity'].get('service_plan_guid', None),
                           'space_guid': resource['entity'].get('space_guid', None)}
                
                if space_lookup is not None and space_lookup.get(service['space_guid'], None) is not None:
                    service['space_name'] = space_lookup[service['space_guid']]['space_name']
                    service['org_name'] = space_lookup[service['space_guid']]['org_name']
                    service['org_guid'] = space_lookup[service['space_guid']]['org_guid']
                        
                if service_lookup is not None and service_lookup.get(service['service_guid'], None) is not None:
                    service['service_name'] = service_lookup[service['service_guid']]
                else:
                    service['service_name'] = None                
                if service_plan_lookup is not None and service_plan_lookup.get(service['service_plan_guid'], None) is not None:
                    service['service_plan_name'] = service_plan_lookup[service['service_plan_guid']]
                else:
                    service['service_plan_name'] = None
                    print('Warning. Found no service plan name for service "{}" plan guid "{}" in org "{}" space "{}"'.format(service['service_instance_name'],
                                                                                                                      service['service_plan_guid'],
                                                                                                                      service['org_name'],
                                                                                                                      service['space_name']))
                service_instances.append(service)
        return response.json()['next_url']  
    else:
        print('Fatal error retrieving service instance information: {}'.format(response))
        return None                

# https://apidocs.cloudfoundry.org/280/service_instances/list_all_service_instances.html
url = '/v2/service_instances?results-per-page=100'

while url is not None:
    print('Loading service instance information...')
    url = fetch_service_instance_list(url)
    
print('Found {} service instances.'.format(len(service_instances)))   



In [ ]:
def fetch_service_keys_list(url):
    http_headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'authorization': authorization
        }
    if debug:
        print('GET {}'.format(API_base_URL.format(url)))
    response = requests.get(API_base_URL.format(url), headers=http_headers)
    if response.status_code == 200:
        for resource in response.json().get('resources', []):
            print(resource)      
        return response.json()['next_url']                
    else:
        print('Fatal error retrieving service key information: {}'.format(response))
        return None

# https://apidocs.cloudfoundry.org/245/service_instances/list_all_service_keys_for_the_service_instance.html
url = '/v2/service_instances/{}/service_keys'.format('d71cb4a5-8345-41bb-83db-b0f3e255ce8e')

while url is not None:
    print('Loading service key information...')
    url = fetch_service_keys_list(url)
    
    

In [ ]:
def fetch_service_instance_credentials_list(service_instance_guid):
    
    http_headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'authorization': authorization
    }

    # https://apidocs.cloudfoundry.org/245/service_instances/list_all_service_keys_for_the_service_instance.html
    url = '/v2/service_instances/{}/service_keys'.format(service_instance_guid)  
    while url is not None:
        print('Loading service instance credentials...') 
        response = requests.get(API_base_URL.format(url), headers=http_headers)
        if response.status_code == 200:
            for resource in response.json().get('resources', []):
                print(resource['entity'])      
            url = response.json()['next_url']                
        else:
            print('Fatal error retrieving service key information: {}'.format(response))
            url = None
    

In [ ]:
import pandas as pd
services_df = pd.DataFrame(service_instances)

Display the DataFrame for debugging purposes

In [ ]:
import pixiedust
display(services_df, debug = True)

### App description:

* by default all provisioned service instance types are displayed in the table
* user selects one service instance type from the drop down
* the service instance list is filtered and displayed


In [ ]:
from pixiedust.display.app import *
import json

@PixieApp
@Logger()
class FetchServiceCredentials():
    
    @route()
    @templateArgs 
    def main_screen(self):
        
        self.debug("Entering method main screen")
        
        self.state = {
            'filter': {
                'service_guid': None,
                'service_plan_name': None,
                'org_guid': None,
                'space_guid': None
            }
        }
        
        self.services_df = services_df.fillna({'service_plan_name':'[UNKNOWN/DISCONTINUED]'})
        self.services_df['hide_service_instance'] = False
        self.service_types_df = self.services_df[['service_guid', 'service_name']].drop_duplicates().sort_values(by=['service_name'])
        return """
            <!-- custom styling -->
            <style>
                div.outer-wrapper {
                    display: table;width:100%;height:100px;
                }
                div.inner-wrapper {
                    display: table-cell;vertical-align: middle;height: 100%;width: 100%;
                }
                th { text-align:center; }
                td { text-align: left; }
            </style>

            <!-- filters -->
            <div class="outer-wrapper" 
                 id="filters{{prefix}}"
                 pd_target="filters{{prefix}}"
                 pd_options="op=display_filters"
                 pd_render_onload
                 class="no_loading_msg">
            </div> 
        
            <!-- service instance list --> 
            <div class="outer-wrapper" 
                 id="matching_service_list{{prefix}}"
                 pd_target="matching_service_list{{prefix}}"
                 pd_options="op=display_service_list"
                 pd_render_onload>
            </div>
        
            <!-- service instance credentials --> 
            <div class="outer-wrapper" id="credentials_list{{prefix}}">

            </div>     
        """

       
    @route(op="display_filters")
    @templateArgs
    def display_filters(self):

        self.info("Entering method display_filters: {} {} {} {}".format(self.state['filter']['service_guid'],
                                                                         self.state['filter']['service_plan_name'],
                                                                         self.state['filter']['org_guid'],
                                                                         self.state['filter']['space_guid']))
        # helpers: temporarily store filter options 
        service_filter_options = {}
        service_plan_filter_options = {}
        org_filter_options = {}
        space_filter_options = {}
    
        for index, row in self.services_df.iterrows():
            service_filter_options[row['service_guid']] = row['service_name']
            if self.state['filter']['service_guid'] is None or self.state['filter']['service_guid'] == row['service_guid']:                
                service_plan_filter_options[row['service_plan_name']] = row['service_plan_name'] 
                if self.state['filter']['service_plan_name'] is None or self.state['filter']['service_plan_name'] == row['service_plan_name']:                
                    org_filter_options[row['org_guid']] = row['org_name']
                    if self.state['filter']['org_guid'] is None or self.state['filter']['org_guid'] == row['org_guid']:                    
                        space_filter_options[row['space_guid']] = row['space_name']                

        # sort filter options using the display name (case-insentitive)
        sorted_service_filter_options_keys = sorted(service_filter_options, key=lambda k: service_filter_options.get(k,'').lower())
        sorted_service_plan_filter_options_keys = sorted(service_plan_filter_options, key=lambda k: service_plan_filter_options.get(k,'').lower())
        sorted_org_filter_options_keys = sorted(org_filter_options, key=lambda k: org_filter_options.get(k,'').lower())
        sorted_space_filter_options_keys = sorted(space_filter_options, key=lambda k: space_filter_options.get(k,'').lower())
        
        self.info('Service plan filter options: {}'.format(service_plan_filter_options))
        self.info('Sorted service plan filter option keys: {}'.format(sorted_service_plan_filter_options_keys))
        self.info('Org filter options: {}'.format(org_filter_options))
        self.info('Sorted org filter option keys: {}'.format(sorted_org_filter_options_keys))
        self.info('Space filter options: {}'.format(space_filter_options))
        self.info('Sorted space filter option keys: {}'.format(sorted_space_filter_options_keys))
        
        # render filters
        return  """
            <select id="service_guid_filter{{prefix}}"
                    pd_script="self.reset_selected_service_guid_filter('$val(service_guid_filter{{prefix}})')"
                    pd_refresh="filters{{prefix}},matching_service_list{{prefix}}"
                    class="no_loading_msg">

               <option value="---ALL---">--- All services ---</option>
             {%for filter_option in sorted_service_filter_options_keys %}             
                {% if this['state']['filter']['service_guid'] == filter_option %}
                    <option value="{{filter_option}}" selected>{{service_filter_options[filter_option]}}</option>
                {% else %}
                    <option value="{{filter_option}}">{{service_filter_options[filter_option]}}</option>
                {%endif %} 
             {%endfor%}
            </select>
            
            <select id="service_plan_filter{{prefix}}" 
                    pd_script="self.reset_selected_service_plan_filter('$val(service_plan_filter{{prefix}})')"
                    pd_refresh="filters{{prefix}},matching_service_list{{prefix}}"
                    class="no_loading_msg">
                <option value="---ALL---">--- All service plans ---</option>
            {% for plan_name in sorted_service_plan_filter_options_keys %}
              {% if this['state']['filter']['service_plan_name'] == service_plan_filter_options[plan_name] %}
                <option value="{{service_plan_filter_options[plan_name]}}" selected>{{service_plan_filter_options[plan_name]}}</option>
              {% else %}
                <option value="{{service_plan_filter_options[plan_name]}}">{{service_plan_filter_options[plan_name]}}</option>
              {%endif %}  
            {% endfor %}                             
            </select>
            
            <select id="org_guid_filter{{prefix}}" 
                     pd_script="self.reset_selected_org_guid_filter('$val(org_guid_filter{{prefix}})')"
                     pd_refresh="filters{{prefix}},matching_service_list{{prefix}}"
                     class="no_loading_msg">
               <option value="---ALL---">--- All organizations ---</option>
              {%for filter_option in sorted_org_filter_options_keys %}             
                {% if this['state']['filter']['org_guid'] == filter_option %}
                    <option value="{{filter_option}}" selected>{{org_filter_options[filter_option]}}</option>
                {% else %}
                    <option value="{{filter_option}}">{{org_filter_options[filter_option]}}</option>
                {%endif %} 
             {%endfor%}              
            </select>
            
            <select id="space_guid_filter{{prefix}}"
                    pd_script="self.reset_selected_space_guid_filter('$val(space_guid_filter{{prefix}})')"
                    pd_refresh="filters{{prefix}},matching_service_list{{prefix}}"
                    class="no_loading_msg">
               <option value="---ALL---">--- All spaces ---</option>
              {%for filter_option in sorted_space_filter_options_keys %}             
                {% if this['state']['filter']['space_guid'] == filter_option %}
                    <option value="{{filter_option}}" selected>{{space_filter_options[filter_option]}}</option>
                {% else %}
                    <option value="{{filter_option}}">{{space_filter_options[filter_option]}}</option>
                {%endif %} 
             {%endfor%}                
            </select>        
        """
   
    @route(op="display_service_list")
    def display_service_list(self):
        
        self.info("Entering method display_service_list: {} {} {} {}".format(self.state['filter']['service_guid'],
                                                                      self.state['filter']['service_plan_name'],
                                                                      self.state['filter']['org_guid'],
                                                                      self.state['filter']['space_guid']))
            
        # define filter function
        def filter_df(r, service_guid = None, service_plan_name = None, org_guid = None, space_guid = None):
            """ Return True if this row should be hidden
            """
            if service_guid is not None and service_guid != r['service_guid']:
                return True
            if service_plan_name is not None and service_plan_name != r['service_plan_name']:
                return True
            if org_guid is not None and org_guid != r['org_guid']:
                return True
            if space_guid is not None and space_guid != r['space_guid']:
                return True
            return False
        
        # apply filter function on DataFrame, setting field _is_hidden to True or False for each row, as appropriate
        self.services_df['hide_service_instance'] = self.services_df.apply(filter_df, 
                                                                           axis = 1, 
                                                                           service_guid = self.state['filter']['service_guid'], 
                                                                           service_plan_name = self.state['filter']['service_plan_name'], 
                                                                           org_guid = self.state['filter']['org_guid'], 
                                                                           space_guid = self.state['filter']['space_guid'])
 
        # Debug: display number of rows that are hidden/not hidden
        self.debug(self.services_df.groupby(by='hide_service_instance').size())

        # render list of services that are not marked as hidden
        return  """
         <table class="table">
           <thead>
             <tr>
                <th>Service Instance Name</th>
                <th>Service Name</th>
                <th>Service Plan</th>
                <th>Organization</th>
                <th>Space</th>
                <th>Actions</th>
             </tr>
          </thead>
          <tbody>
          {% for row in this.services_df.sort_values(by=['service_instance_name']).itertuples()%}
           {% if row['hide_service_instance'] == False %}
            <tr>
                <td>{{row['service_instance_name']}}</td>
                <td>{{row['service_name']}}</td>
                <td>{{row['service_plan_name']}}</td>
                <td>{{row['org_name']}}</td>
                <td>{{row['space_name']}}</td>
                <td><button class="btn btn-default" type="button" pd_options="service_instance_guid={{row['service_instance_guid']}}" pd_target="credentials_list{{prefix}}">View credentials</button></td>
            </tr>
           {% endif %}
          {% endfor %}
          </tbody>
         </table>
        """
    
    @route(service_instance_guid="*")
    @templateArgs
    def list_credentials(self, service_instance_guid):
        
        self.debug("Entering method list_credentials: {}".format(service_instance_guid))
            
        http_headers = {
            'accept': 'application/json',
            'content-type': 'application/json',
            'authorization': authorization
        }

        service_instance_credentials = []    
        
        # https://apidocs.cloudfoundry.org/245/service_instances/list_all_service_keys_for_the_service_instance.html
        url = '/v2/service_instances/{}/service_keys'.format(service_instance_guid)  
        while url is not None:
            print('Loading service instance credentials...') 
            response = requests.get(API_base_URL.format(url), headers=http_headers)
            if response.status_code == 200:
                for resource in response.json().get('resources', []):
                    service_instance_credentials.append({"name":resource['entity']['name'] , 
                                                         "credentials": resource['entity']['credentials'],
                                                         "formatted_credentials": json.dumps(resource['entity']['credentials'], indent=4)})     
                url = response.json()['next_url']                
            else:
                print('Fatal error retrieving service key information: {}'.format(response))
                url = None

 
        # Debug: display credentials
        self.debug(service_instance_credentials)        
        
        if len(service_instance_credentials) == 0:
            return """
            <h2>There are no service credentials defined for this instance</h2>
            """
        else:     
                     
            # render credentials list 
            return  """
              <h2>Credentials for service</h2>
              <div>
              <ul class="list-group">
              {% for credential in service_instance_credentials %}
                
                <li class="list-group-item">
                  <button class="btn btn-default" 
                          type="button">
                    <pd_script>
get_ipython().set_next_input("# @hidden_cell\\n{}={}".format("credentials", json.dumps({{credential['credentials']}}, indent=4)))
                    </pd_script>
                 Copy "{{credential['name']}}" into notebook cell</button>               
                </li>
                <!-- <div><pre>credentials={{credential['formatted_credentials']}}</pre> -->
              {% endfor %} 
              </ul>
              </div>
              
            """
    
    def reset_selected_service_guid_filter(self, service_guid=None):
        """
        """
        self.info("Resetting service guid filter and its dependencies to {}".format(service_guid))
        if service_guid == '---ALL---':
            service_guid = None
            
        self.state['filter'] = {
            'service_guid': service_guid,
            'service_plan_name': None,
            'org_guid': None,
            'space_guid': None
        }
        return
    
    def reset_selected_service_plan_filter(self, service_plan_name=None):
        """
        """
        self.info("Resetting service plan filter and its dependencies to {}".format(service_plan_name))
        if service_plan_name == "---ALL---":
            service_plan_name = None
            
        self.state['filter']['service_plan_name'] = service_plan_name
        self.state['filter']['org_guid'] = None
        self.state['filter']['space_guid'] = None
        return
  
    def reset_selected_org_guid_filter(self, org_guid=None):
        """
        """
        self.info("Resetting org guid filter and its dependencies to {}".format(org_guid))
            
        if org_guid == "---ALL---":
            org_guid = None
        
        self.state['filter']['org_guid'] = org_guid
        self.state['filter']['space_guid'] = None
        return

    def reset_selected_space_guid_filter(self, space_guid=None):
        """
        """
        self.info("Resetting space guid filter and its dependencies to {}".format(space_guid))
        if space_guid == "---ALL---":
            space_guid = None
            
        self.state['filter']['space_guid'] = space_guid
        return    
    
    
# --------------------------------------------------------
# instantiate app and run it
app = FetchServiceCredentials()
app.run()


# DEBUG log